# ABBA cell count analysis

This notebook is the last step in the ABBA whole-brain cell counting analysis.  
It assumes you have done the following steps:
- Alignment of brain slices in ABBA, exported to a QuPath project.
- Detected cells of interest in QuPath. The detections should be exported to ```.csv``` files (one per slice) in a folder called ```results```. 
- If there are regions to exclude, you should have drawn them and exported to ```.txt``` files (one per slice) in a folder called ```regions_to_exclude```.

Run this notebook to load the cell counts and do analysis on them. 

## Before we start ...
### Set parameters

In [ ]:
import os

CONFIG_FILE_NAME = "braian_config.toml"                     # assumes the file is in DATA_ROOT directory
EXPERIMENT_DIRECTORY = "ieg"
EXPERIMENT_DIRECTORY = "cohort4"

USE_REMOTE_DATA = False                                     # if True, it tries to read the data on the laboratory's server
# ###################################### REMOTE DIRECTORIES #####################################
IS_COLLABORATION_PROJ = False
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
# ###################################### PLOT OPTIONS ######################################
PLOT_ALLENBRAIN_HIERARCHY = False
PLOT_ANIMALS_ROOTS = True
PLOT_COEFFICIENT_OF_VARIATION = True
PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD = 1

REMOVE_SMALL_REGIONS_FROM_SLICES = False
REMOVE_HIGH_CV_REGIONS = False
SAVE_ANIMALS = True
SAVE_GROUPS = True

# Script's code

In [ ]:
import os
import sys
from typing import List

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if USE_REMOTE_DATA:
    DATA_ROOT, _ = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, IS_COLLABORATION_PROJ, COLLABORATION_DIRECTORY)

data_input_path = os.path.join(DATA_ROOT, "QuPath_output")
data_output_path= os.path.join(DATA_ROOT, "BraiAn_output")
plots_output_path = PLOTS_ROOT
config_file = os.path.join(DATA_ROOT, CONFIG_FILE_NAME)

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)
if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
import tomllib

with open(config_file, "rb") as f:
    config = tomllib.load(f)
config
# ######################################### LOAD CONFIG #########################################
EXPERIMENT_NAME = config["experiment"]["name"]

ATLAS_VERSION = config["atlas"]["version"]
BRANCHES_TO_EXCLUDE = config["atlas"]["excluded-branches"]
USE_LITERATURE_REUNIENS =  config["atlas"]["use-literature-reuniens"]   # add a 'REtot' region, merging the following regions: 'RE', 'Xi', 'RH'

BRAINS_AREA_KEY = config["brains"]["area-column"]
BRAINS_TRACER_KEYS = config["brains"]["tracer-columns"]
BRAINS_MARKERS = config["brains"]["markers"]
BRAINS_AGGREGATION_MODE = config["brains"]["slices-aggregation-mode"]   # available options are: 'sum', 'mean'/'avg', 'std', 'variation'/'cvar'

from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "id name dirs")
groups = [
    GroupDirectory(
        id=int(group[len("group"):])-1,
        name=config["experiment"][group]["name"],
        dirs=config["experiment"][group]["dirs"]
    ) for group in config["experiment"] if group.startswith("group") and group[len("group"):].isdigit()
]

## The Allen Brain Atlas

We start by importing the mouse Allen Brain Atlas, in which we find information about all brain regions (their parent region and children regions in the brain hierarchy, for example).

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, use_literature_reuniens=USE_LITERATURE_REUNIENS, version=ATLAS_VERSION)
# path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
# AllenBrain.select_from_csv(path_to_summary_structures, include_nre_tot=USE_LITERATURE_REUNIENS)
# AllenBrain.select_at_depth(8)
AllenBrain.select_leaves()
selected_regions = AllenBrain.get_selected_regions()
print(f"You selected {len(selected_regions)} Summary Structure regions.")

#parent_region = AllenBrain.parent_region
#direct_subregions = AllenBrain.direct_subregions
#full_name = AllenBrain.full_name
#regions = AllenBrain.list_all_subregions("root", mode="depth")

We can also visualize the hierarchy of brain regions as a network (a tree). **Note that running the above cell may take a few minutes**.

In [ ]:
## Plot brain region hierarchy
## If you want to plot it, install PyDot (pydot)
if PLOT_ALLENBRAIN_HIERARCHY:
    fig = AllenBrain.plot_plotly_graph()
    fig.show()

Based on the graph above, you might want to specify the regions on which you want to do further PLS analysis:  
*Note: to see more information about the regions, hover over them with your mouse.*

- Specify a level. Analysis can only be done on one level (slice) in the brain region.

- To exclude brain regions that belong to a certain branch, add the *abbreviated* nodes at the beginning of the branches to the list above.  
Example:  
```branches_to_exclude = ["retina", "VS"]```  
means that **all the subregions that belong to the retina and the ventricular systems** are excluded from the PLS analysis.

## Load data

Now, we're ready to read the ```.csv``` files with the cell counts, and also the exclusion files (if there were regions to exclude).  
Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_dirs```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_dirs```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```area_key```: A string of the column in the ```.csv``` files that refers to the size of a brain areatra
- ```tracer_key```: A string of the column in the ```.csv``` files that refers to the tracer number used to highlight the marker
- ```marker```: A string of the marker we would like to highlight (e.g. CFos)

Provare a modificar per ottenere densita in mm^2 (da micron)

Now, we load the Control and Stress results seperately in two pandas dataframes, and save the results.

**Note**: regions to exclude are automatically excluded.

In [ ]:
import re
# retrieve the name of the marker of the channel used as control for the overlapping
marker1 = next(s for s in BRAINS_MARKERS if re.match("[a-zA-Z0-9]+-\(.*\)", s)).split("-(")[0]
marker2 = min(BRAINS_MARKERS, key=len)
marker1_diff = f"{marker1}-({marker1}+{marker2})"

In [ ]:
groups_slices: List[List[BraiAn.SlicedBrain]] = []
for group in groups:
    sliced_brains = []
    for animal_dir in group.dirs:
        if not os.path.isdir(os.path.join(data_input_path, animal_dir)):
            print(f"WARNING: could not find the directory '{animal_dir}' in '{EXPERIMENT_DIRECTORY}'. Skipping this animal.")
            continue
        multich_sliced_brain = BraiAn.SlicedBrain(animal_dir,
                                                os.path.join(data_input_path, animal_dir),
                                                AllenBrain,
                                                BRAINS_AREA_KEY,
                                                BRAINS_TRACER_KEYS,
                                                BRAINS_MARKERS,
                                                area_units="µm2")
        for i in range(len(multich_sliced_brain.slices)):
            multich_sliced_brain.slices[i].data[marker1_diff] += multich_sliced_brain.slices[i].data[f"{marker1}+{marker2}"]
            multich_sliced_brain.slices[i].data.rename(columns={marker1_diff: marker1}, inplace=True)
            multich_sliced_brain.markers = [marker1 if m == marker1_diff else m for m in multich_sliced_brain.markers]
        sliced_brains.append(multich_sliced_brain)
    groups_slices.append(sliced_brains)
    print(f"Imported all brain slices from {str(len(group.dirs))} animals of {group.name} group.")

In [ ]:
if REMOVE_SMALL_REGIONS_FROM_SLICES:
    for group_ in groups_slices:
        for animal_ in group_:
            for s in animal_.slices:
                s._data = s.data
                s.data = s.data[(s.data[marker1] != 1) & (s.data[marker2] != 1) & (s.data.area > 0.001)].copy(deep=True)

In [ ]:
# 2023_08_28__0792.czi - Scene #08 of 371 excludes both right and left of 'root'!
if PLOT_ANIMALS_ROOTS:
    region_name = "root"
    root_plot = BraiAn.plot_region_density(region_name, *groups_slices, width=1000, height=500)
    root_plot.layout.title = f"IEG densities in '{region_name}'"
    root_plot.show()

In [ ]:
# print("N regions above threshold:", sum([(brain.data > cv_threshold).sum() for brain in cvar_brains]))
# print("N regions below threshold:", sum([(brain.data <= cv_threshold).sum() for brain in cvar_brains]))
if PLOT_COEFFICIENT_OF_VARIATION:
    cvar_plot = BraiAn.plot_cv_above_threshold(AllenBrain, *groups_slices, cv_threshold=PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD, width=1000, height=500)
    cvar_plot.show()

In [ ]:
r = "MD"
a = "371"
# m = "cFos"
def check_animal_region(animal_name: str, region_acronym: str, marker=None):
    try:
        sliced_brain = next(animal for group in groups_slices for animal in group if animal_name == animal.name)
    except StopIteration:
        print(f"Can't find region '{region_acronym}' for animal '{animal_name}'")
        return
    sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
    all_slices_df = sliced_brain.concat_slices()
    slices_per_area = all_slices_df.groupby(all_slices_df.index).count().iloc[:,0]
    markers = sliced_brain.markers if marker is None else [marker]
    brain_avg = BraiAn.AnimalBrain(sliced_brain, mode="avg", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data
    brain_std = BraiAn.AnimalBrain(sliced_brain, mode="std", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data
    for m in markers:
        marker_avg = brain_avg[f"{m}_density"]
        marker_std = brain_std[f"{m}_density"]
        print(f"""Summary for brain region '{region_acronym}' of marker '{m}':
            - N slices: {slices_per_area[region_acronym]}
            - Mean: {marker_avg[region_acronym]:.2f} {m}/mm²),
            - S.D.: {marker_std[region_acronym]:.2f} {m}/mm²,
            - Coefficient of Variation: {marker_avg[region_acronym]}
        """)
# for a in groups[-1].dirs:
check_animal_region(a, r) #, m)

In [ ]:
# if you want to see the slices in specific, run this
import pandas as pd
slices = []
try:
    sliced_brain = next(animal for group in groups_slices for animal in group if a == animal.name)
    sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
    for slice in sliced_brain.slices:
        if r not in slice.data.index:
            continue
        region = slice.data.loc[r].copy()
        for marker in sliced_brain.markers:
            region[f"{marker} density"] = region[marker] / region["area"]
        region["name"] = slice.name
        slices.append(region)
except StopIteration:
    print(f"Can't find region '{r}' for animal '{a}'")
pd.concat(slices, axis=1) if len(slices) != 0 else None

In [ ]:
# This cell:
#  * creates a summed AnimalBrain for each SlicedBrain
#  * from each animal, removes all regions that have a coefficent of variation above cvar_threshold
#  * for each animal, computes the percentage of both marker1 and marker2 overlapping with each other
#  * for each group, computes the normalizations (Density, RelativeDensity & Percentage)
#  * for each group, computes the average percentage of overlapping

import pandas as pd

cvar_threshold = 1

avg_overlap_groups = []
normalized_groups = []
for i, group in enumerate(groups_slices):
    animal_brains_df = []
    animal_brains = []
    for slices in group:
        if REMOVE_HIGH_CV_REGIONS:
            # remove all regions that have a Coefficent of Variation above cvar_threshold
            cvars = BraiAn.AnimalBrain(slices, mode="cvar", hemisphere_distinction=False, min_slices=0).data
            disperse_regions = cvars.index[(cvars > cvar_threshold)[[f"{marker1}_density", f"{marker2}_density"]].any(axis=1)]
            print(f"removing {len(disperse_regions)}/{len(cvars)} dispersive regions from '{slices.name}'")
        animal_brain = BraiAn.AnimalBrain(slices, hemisphere_distinction=False, mode="sum", min_slices=0)
        animal_brains.append(animal_brain)

        animal_brain_df = animal_brain.data
        if REMOVE_HIGH_CV_REGIONS:
            animal_brain_df[~animal_brain_df.index.isin(disperse_regions)]
        animal_brain_df[f"%{marker1}_overlapping"] = animal_brain_df[f"{marker1}+{marker2}"] / animal_brain_df[marker1]
        animal_brain_df[f"%{marker2}_overlapping"] = animal_brain_df[f"{marker1}+{marker2}"] / animal_brain_df[marker2]
        animal_brains_df.append(animal_brain_df)

    normalized_group = BraiAn.AnimalGroup(groups[i].name, animal_brains, AllenBrain)
    normalized_groups.append(normalized_group)
    avg_overlap = dict()
    for i, brain in enumerate(animal_brains_df):
        avg_overlap[i] = brain[[f"%{marker1}_overlapping", f"%{marker2}_overlapping"]]
    avg_overlap = pd.concat(avg_overlap, join="outer").groupby(level=1).mean()
    avg_overlap_groups.append(avg_overlap)

In [ ]:
# animal_brain.to_csv("FC_369.csv", sep=",")
# pd.read_csv("FC_369.csv", sep=",", index_col=0)

In [ ]:
import bgheatmaps as bgh
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import vedo as vd

# vd.embedWindow(None)
vd.embedWindow("k3d") # doesn't spawn a 3rd party window frame for the scene
# vd.embedWindow("itkwidgets")

def make_filename(*ss: str):
    return "_".join((s.replace(' ', '_') for s in ss if s != ""))

def plot_brain(plot_filename: str, brain_data: pd.DataFrame, n: int, title: str, selected_regions: list[str],
                cmin=None, cmax=None, cmap="OrRd", show_text=True):
    heatmaps = []
    hems = ("right", "left")
    # brain_data = brain_data.loc[selected_regions]
    brain_data = brain_data[brain_data.index.isin(selected_regions)]
    brain_data = brain_data[~brain_data.isna().all(axis=1)]
    if cmin is None:
        cmin = math.floor(brain_data[brain_data != np.inf].min(axis=None))
    if cmax is None:
        cmax = math.ceil(brain_data[brain_data != np.inf].max(axis=None))
    for metric, hem in zip(brain_data.columns, hems):
        metric_data = brain_data[metric]
        dataDict = metric_data.to_dict()

        print(f"Rendering '{metric}' heatmap ({hem} hemisphere)...")
        heatmap = bgh.heatmap(
            dataDict,
            position=5000,
            orientation="frontal",
            thickness=15000,
            title=metric,
            cmap=cmap, # "PuBu",
            vmin=cmin,
            vmax=cmax,
            format="2D",
            hemisphere=hem  # this option was added by me in my local installation of bgheatmaps.heatmap() in order to split the heatmap in two
                            # if you don't know how to do it, just remove this row
        )
        #heatmap.scene.close()
        heatmaps.append(heatmap)
    heatmap1, heatmap2 = heatmaps

    print("depths: ", end="")
    for depth in np.linspace(1500,11000,n):
        # center = heatmap.scene.root.centerOfMass()
        print(f"{depth:.2f}", end="  ")
        s = bgh.slicer.Slicer(depth, "frontal", 100, heatmap1.scene.root) # BraiAn.Plane(center, np.array([0,0,1]), np.array([0,1,0])) # frontal
        heatmap.slicer = s
        projected1, _ = heatmap.slicer.get_structures_slice_coords(heatmap1.regions_meshes, heatmap1.scene.root)
        projected2, _ = heatmap.slicer.get_structures_slice_coords(heatmap2.regions_meshes, heatmap2.scene.root)

        f, ax = plt.subplots(figsize=(9, 9))
        for r, coords in projected1.items():
            name, segment = r.split("_segment_")
            filled_polys = ax.fill(
                coords[:, 0],
                coords[:, 1],
                color=heatmap1.colors[name],
                label=name if segment == "0" and name != "root" else None,
                lw=1,
                ec="k",
                zorder=-1 if name == "root" or heatmap1.colors[name] == [0,0,0] else None,
                alpha=0.3 if name == "root" or heatmap1.colors[name] == [0,0,0] else None,
            )
            if name == "root":
                continue
            (x0, y0), (x1, y1) = filled_polys[0].get_path().get_extents().get_points()
            if show_text:
                ax.text((x0 + x1) / 2, (y0 + y1) / 2, name, ha="center", va="center", fontsize=10, color="black")
        for r, coords in projected2.items():
            name, segment = r.split("_segment_")
            filled_polys = ax.fill(
                coords[:, 0],
                coords[:, 1],
                color=heatmap2.colors[name],
                label=name if segment == "0" and name != "root" else None,
                lw=1,
                ec="k",
                zorder=-1 if name == "root" or heatmap2.colors[name] == [0,0,0] else None,
                alpha=0.3 if name == "root" or heatmap2.colors[name] == [0,0,0] else None,
            )
            if name == "root":
                continue
            (x0, y0), (x1, y1) = filled_polys[0].get_path().get_extents().get_points()
            if show_text:
                ax.text((x0 + x1) / 2, (y0 + y1) / 2, name, ha="center", va="center", fontsize=10, color="white")

        # set title
        ax.set_title(title, fontsize=20, pad=-15)
        for data_column, hem in zip(brain_data.columns, reversed(hems)): # the hemispheres are flipped because the brain is cut front->back, not back->front
            ax.set_title(data_column, loc=hem, y=0, pad=-15)

        # style axes
        ax.invert_yaxis()
        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set(xlabel='', ylabel='')
        ax.set_aspect('equal',adjustable='box')

        # add colorbar
        ax.figure.colorbar(
            mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=heatmap.vmin, vmax=heatmap.vmax), cmap=heatmap.cmap),
            ax=ax, label=title, fraction=0.046, pad=0.04
        )

        plot_filepath = os.path.join(plots_output_path, plot_filename+f"_{depth:05.0f}.svg")
        f.savefig(plot_filepath)
        plt.close(f)
    print("")

In [ ]:
selected_regions.remove("RSPd4")

In [ ]:
# plot overlapping
for avg_overlap, group in zip(avg_overlap_groups, groups):
    filename = make_filename("overlapping", group.name, f"{marker1}+{marker2}")
    plot_brain(filename, avg_overlap, 12, "overlapping", selected_regions, cmap="magma_r", show_text=False)

In [ ]:
if len(normalized_groups)%2 == 0:
    for right_g, left_g in zip(normalized_groups[::2], normalized_groups[1::2]):
        for marker in (marker1, marker2):
            marker_density_change = pd.concat({
                    right_g.name: right_g.group_by_region(marker=marker, method="Density").mean(), # .apply(lambda x: np.nan if x.isna().all() else np.nanmean(x)), # .data[marker]["Density"].groupby(level=0).mean(),
                    left_g.name:   left_g.group_by_region(marker=marker, method="Density").mean()  # .apply(lambda x: np.nan if x.isna().all() else np.nanmean(x))  # .data[marker]["Density"].groupby(level=0).mean()
                }, axis=1)
            # filename = make_filename("density", "hsv", marker, f"{left_g.name}+{right_g.name}")
            # plot_brain(filename, marker_density_change, 12, "density", selected_regions, cmap="hsv")
            filename = make_filename("density", marker, f"{right_g.name}+{left_g.name}")
            plot_brain(filename, marker_density_change, 12, "density", selected_regions, cmap="magma_r", show_text=False)

In [ ]:
if len(normalized_groups) == 2:
    # for each marker, get the density fold change of group1 (e.g. FC) over group2 (e.g. HC)
    fold_change = dict()
    for marker in (marker1, marker2):
        marker_density_change = pd.concat({
                normalized_group.name: normalized_group.data[marker]["Density"].groupby(level=0).mean()
                for normalized_group in normalized_groups
            }, axis=1)
        # compute the fold change of group1 over group2
        fold_change[f"{marker}_density_fold_change"] = marker_density_change[normalized_groups[0].name] / marker_density_change[normalized_groups[1].name]
    fold_change = pd.concat(fold_change, axis=1)
    # filename = make_filename("fold change", "hsv", f"{marker1}+{marker2}")
    # plot_brain(filename, fold_change, 12, "fold change", selected_regions, cmap="hsv")
    filename = make_filename("fold change", "max5", f"{marker1}+{marker2}")
    plot_brain(filename, fold_change, 12, "fold change", selected_regions, cmin=-3, cmax=5, cmap="coolwarm")
               

In [ ]:
import importlib
__imported_modules = sys.modules.copy()
for module_name, module in __imported_modules.items():
    if not module_name.startswith("BraiAn"):
        continue
    try:
        importlib.reload(module)
    except ModuleNotFoundError:
        continue